In [42]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [59]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split,KFold,cross_validate
from sklearn.metrics import mean_absolute_error,mean_squared_error,mean_absolute_percentage_error
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np



<IPython.core.display.Javascript object>

In [31]:
def one_hot_encoding(objs: list, df) -> pd.DataFrame:
    encoder = OneHotEncoder()
    one_hots = []
    for obj in objs:
        one_hot = encoder.fit_transform(df[obj].values.reshape(-1, 1)).toarray()
        one_hots.append(
            pd.DataFrame(
                one_hot, columns=[f"{obj}_{new}" for new in encoder.categories_[0]]
            )
        )
    return pd.concat(one_hots, axis=1)

<IPython.core.display.Javascript object>

In [47]:
# load data
auto = pd.read_csv(r"Automobile_Price.csv")

# drop na price
auto = auto.dropna(subset=["price"])
auto_data = auto.drop(["price", "symboling", "normalized-losses"], axis=1)

auto_price = auto["price"]

# impute

impute_df = auto_data.copy()
objects = impute_df.select_dtypes("object").columns.tolist()
impute_df = impute_df.drop(objects, axis=1)
imputer = SimpleImputer(strategy="most_frequent")
impute_df = pd.DataFrame(imputer.fit_transform(impute_df), columns=impute_df.columns)

# one hot encoding

result_one_hot = one_hot_encoding(objects, auto_data)
encoding_df = pd.concat([impute_df, result_one_hot], axis=1)


target = auto_price
data = encoding_df

<IPython.core.display.Javascript object>

In [57]:
target.head()

0    13495.0
1    16500.0
2    16500.0
3    13950.0
4    17450.0
Name: price, dtype: float64

<IPython.core.display.Javascript object>

In [56]:
result_one_hot.head()

,make_alfa-romero,make_audi,make_bmw,make_chevrolet,make_dodge,make_honda,make_isuzu,make_jaguar,make_mazda,make_mercedes-benz,...,num-of-cylinders_twelve,num-of-cylinders_two,fuel-system_1bbl,fuel-system_2bbl,fuel-system_4bbl,fuel-system_idi,fuel-system_mfi,fuel-system_mpfi,fuel-system_spdi,fuel-system_spfi
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


<IPython.core.display.Javascript object>

In [60]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.3, random_state=2022
)
lr = LinearRegression()
lr.fit(X_train, y_train)
predict = lr.predict(X_test)
r2 = lr.score(X_test, y_test)
mae = mean_absolute_error(y_test, predict)
mse = mean_squared_error(y_test, predict)
rmse = mean_squared_error(y_test, predict, squared=False)
mape = mean_absolute_percentage_error(y_test,predict)
print(f"{r2=:.2f}, {mae=:.2f}, {mse=:.2f}, {rmse=:.2f},{mape=:.2f}")

r2=0.83, mae=1910.80, mse=8292242.01, rmse=2879.63,mape=0.17


<IPython.core.display.Javascript object>

In [64]:
# k fold
kfold = KFold(n_splits=9)

r2_list=[]
mae_list=[]
mse_list=[]
rmse_list=[]

for train_idx,test_idx in kfold.split(data):

    X_train,X_test = data.iloc[train_idx.tolist()],data.iloc[test_idx.tolist()]
    y_train,y_test = target.iloc[train_idx.tolist()],target.iloc[test_idx.tolist()]
    
    lr.fit(X_train,y_train)
    r2_list.append(lr.score(X_test,y_test))
    predict = lr.predict(X_test)
    mae_list.append( mean_absolute_error(y_test,predict))
    mse_list.append( mean_squared_error(y_test,predict))
    rmse_list.append (mean_squared_error(y_test,predict,squared=False))

print(f'{r2_list=}\n{mae_list=}\n{mse_list=}\n{rmse_list=}\n')
print(f'r2: {np.mean(r2_list):.2f}')
print(f'mae: {np.mean(mae_list):.2f}')
print(f'mse: {np.mean(mse_list):.2f}')
print(f'rmse: {np.mean(rmse_list):.2f}')

InvalidIndexError: (slice(None, None, None), array([ 23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
        36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
        49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
        62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,
        75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
        88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
       101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
       140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
       153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
       166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
       179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
       192, 193, 19

<IPython.core.display.Javascript object>

In [62]:
# cross validate

score = cross_validate(
    lr,
    data,
    target,
    cv=9,
    scoring=[
        "r2",
        "neg_mean_absolute_error",
        "neg_mean_squared_error",
        "neg_root_mean_squared_error",
    ],
)
print(score)
print(f'r2: {abs(score.get("test_r2").mean()):.2f}')
print(f'mae: {abs(score.get("test_neg_mean_absolute_error").mean()):.2f}')
print(f'mse: {abs(score.get("test_neg_mean_squared_error").mean()):.2f}')
print(f'rmse: {abs(score.get("test_neg_root_mean_squared_error").mean()):.2f}')

{'fit_time': array([0.00383425, 0.00562978, 0.00580907, 0.00251508, 0.00229192,
       0.0027411 , 0.00405216, 0.00441504, 0.00219607]), 'score_time': array([0.001513  , 0.00126624, 0.00117803, 0.00112796, 0.00082421,
       0.00076485, 0.00102401, 0.00081205, 0.00087094]), 'test_r2': array([ 0.39179198,  0.88224457,  0.85087949,  0.88987078,  0.42503427,
        0.3707909 , -1.13810271, -1.99874464,  0.77839601]), 'test_neg_mean_absolute_error': array([-5174.79303631, -2139.7950191 , -3187.64839808, -3049.15006604,
       -2473.8335743 , -5333.09185802, -1962.86467488, -4040.5771787 ,
       -1732.31469931]), 'test_neg_mean_squared_error': array([-53620277.46103159,  -6597879.04707342, -13628662.54807381,
       -14141664.95270403,  -8551009.0324353 , -51494406.38986761,
        -5088453.04524283, -27452901.84282067,  -4924519.5637013 ]), 'test_neg_root_mean_squared_error': array([-7322.58680119, -2568.63369266, -3691.70184983, -3760.54051337,
       -2924.21083926, -7175.96031134, -2

<IPython.core.display.Javascript object>

In [48]:
import numpy as np

CATEGORY_TYPE = [
    str, object,
    np.str, np.object
]
BOOLEAN_TYPE = [
    bool,
    np.bool_,
]
NUMBER_TYPE = [
    int, float,
    np.uint8, np.uint16, np.uint32, np.uint64,
    np.int_, np.int64, np.int32, np.int16, np.int8,
    np.float_, np.float64, np.float32, np.float16
]
COMPLEX_TYPE = [
    complex,
    np.complex_, np.complex64, np.complex128
]
NULL_TYPE = [
    None,
    np.nan
]

METRIC_TYPES = {
    "regression": ["mae", "rmse", "ndcg", "mape", "mse", "cod"],
    "classification": ["f1", "auc", "recall", "accuracy", "precision"],
    "recommendation": ["mae", "rmse", "ndcg", "mape", "mse", "cod"]
}

TRAIN_CONSTANCE = {
    "coverage": "FULL",
    "sampling": "STD",
    "algorithm": "autoML",
    "ratio": 0.3,
    "type": "regression"
}

/var/folders/61/f3xnht9d2mg0ncd3k019b9j80000gn/T/ipykernel_29958/3724788345.py:5: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.str, np.object
/var/folders/61/f3xnht9d2mg0ncd3k019b9j80000gn/T/ipykernel_29958/3724788345.py:5: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.str, np.object


<IPython.core.display.Javascript object>

In [49]:

from typing import Tuple

def feature_extract(df: pd.DataFrame) -> Tuple[dict, dict]:
    dataset_formats, missing_rates = {}, {}
    for col_name in df:
        if df[col_name].dtype in COMPLEX_TYPE:
            raise ValueError("Not support Imaginary Number")
        elif df[col_name].dtype in BOOLEAN_TYPE:
            dataset_formats[col_name] = "T/F"
        elif df[col_name].dtype in NUMBER_TYPE:
            dataset_formats[col_name] = "NUMBER"
        elif df[col_name].dtype in CATEGORY_TYPE:
            switch = True
            for data in df[col_name]:
                if type(data) in NULL_TYPE or pd.isna(data):
                    continue
                elif type(data) in NUMBER_TYPE:
                    switch = False
                    dataset_formats[col_name] = "불일치"
                    break
            if switch:
                dataset_formats[col_name] = "STRING"
        elif df[col_name].dtype in NULL_TYPE:
            dataset_formats[col_name] = "빈 컬럼"
    row_count = df.shape[0]
    if row_count == 0:
        missing_rates = {col: 0.0 for col in df}
    else:
        for col_name in df:
            miss_count = df[col_name].isna().sum()
            missing_rates[col_name] = miss_count / row_count
    return dataset_formats, missing_rates



<IPython.core.display.Javascript object>

In [51]:
dataset_formats,missing_rates=feature_extract(auto)

<IPython.core.display.Javascript object>

In [52]:
dataset_formats

{'symboling': 'NUMBER',
 'normalized-losses': 'NUMBER',
 'make': 'STRING',
 'fuel-type': 'STRING',
 'aspiration': 'STRING',
 'num-of-doors': 'STRING',
 'body-style': 'STRING',
 'drive-wheels': 'STRING',
 'engine-location': 'STRING',
 'wheel-base': 'NUMBER',
 'length': 'NUMBER',
 'width': 'NUMBER',
 'height': 'NUMBER',
 'curb-weight': 'NUMBER',
 'engine-type': 'STRING',
 'num-of-cylinders': 'STRING',
 'engine-size': 'NUMBER',
 'fuel-system': 'STRING',
 'bore': 'NUMBER',
 'stroke': 'NUMBER',
 'compression-ratio': 'NUMBER',
 'horsepower': 'NUMBER',
 'peak-rpm': 'NUMBER',
 'city-mpg': 'NUMBER',
 'highway-mpg': 'NUMBER',
 'price': 'NUMBER'}

<IPython.core.display.Javascript object>

In [54]:
missing_rates

{'symboling': 0.0,
 'normalized-losses': 0.18407960199004975,
 'make': 0.0,
 'fuel-type': 0.0,
 'aspiration': 0.0,
 'num-of-doors': 0.009950248756218905,
 'body-style': 0.0,
 'drive-wheels': 0.0,
 'engine-location': 0.0,
 'wheel-base': 0.0,
 'length': 0.0,
 'width': 0.0,
 'height': 0.0,
 'curb-weight': 0.0,
 'engine-type': 0.0,
 'num-of-cylinders': 0.0,
 'engine-size': 0.0,
 'fuel-system': 0.0,
 'bore': 0.01990049751243781,
 'stroke': 0.01990049751243781,
 'compression-ratio': 0.0,
 'horsepower': 0.009950248756218905,
 'peak-rpm': 0.009950248756218905,
 'city-mpg': 0.0,
 'highway-mpg': 0.0,
 'price': 0.0}

<IPython.core.display.Javascript object>